1. Get the HTML

In [1]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pandas as pd 
df_list = pd.read_html('https://en.wikipedia.org/wiki/Mitch_McConnell')

2.Get the infobox 

The CSS selector of the infobox is class = infobox vcard.

In [3]:
import requests
from bs4 import BeautifulSoup

response = requests.get("https://en.wikipedia.org/wiki/Mitch_McConnell")
content = response.content
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
infobox = soup.find_all(class_="infobox")

3. Make a dataframe

In [5]:
all_keys = [a.text for a in soup.find_all(class_ ="infobox-label")]
all_keys.insert(0, 'Full Name')

full_name = [a.text for a in soup.find_all(class_ = "fn")]
all_values = [a.text for a in soup.find_all(class_ ="infobox-data")]
all_values.insert(0, full_name[0])

In [6]:
all_keys[0:5]

['Full Name', 'Whip', 'Preceded by', 'Whip', 'Preceded by']

In [7]:
all_values[0:5]

['Mitch McConnell',
 'John Thune',
 'Chuck Schumer',
 'Trent LottJon KylJohn Cornyn',
 'Harry Reid']

In [8]:
df = pd.DataFrame({'key': all_keys, 'value':all_values})
df.head()

,key,value
0,Full Name,Mitch McConnell
1,Whip,John Thune
2,Preceded by,Chuck Schumer
3,Whip,Trent LottJon KylJohn Cornyn
4,Preceded by,Harry Reid


In [9]:
newdf = df[df.key.isin(["Full Name", "Political party","Children"])]

In [10]:
newdf

,key,value
0,Full Name,Mitch McConnell
24,Political party,Republican
26,Children,3


4. Make a function

In [11]:
def get_wiki_info(url):
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    all_keys = [a.text for a in soup.find_all(class_ ="infobox-label")]
    all_keys.insert(0, 'Full Name')
    
    full_name = [a.text for a in soup.find_all(class_ = "fn")]
    all_values = [a.text for a in soup.find_all(class_ ="infobox-data")]
    all_values.insert(0, full_name[0])
    
    if "Children" in all_keys:
        pass
    else:
        all_keys.append('Children')
        all_values.append('NA')
        
    df = pd.DataFrame({'key': all_keys, 'value':all_values})
    newdf = df[df.key.isin(["Full Name", "Political party","Children"])]
    return newdf

In [12]:
get_wiki_info('https://en.wikipedia.org/wiki/Tammy_Duckworth')

,key,value
0,Full Name,Tammy Duckworth
13,Political party,Democratic
15,Children,2


In [13]:
get_wiki_info('https://en.wikipedia.org/wiki/Susan_Collins')

,key,value
0,Full Name,Susan Collins
7,Political party,Republican
13,Children,NA


5. Get all senators' pages

In [14]:
all_request = requests.get("https://en.wikipedia.org/wiki/List_of_current_United_States_senators")
all_content = all_request.content
all_soup = BeautifulSoup(all_request.content, 'html.parser')

In [15]:
vcard = all_soup.find_all(class_="vcard")

In [16]:
all_senators = [a.a.get('href') for a in all_soup.find_all(class_="fn")]

In [17]:
all_senators

['/wiki/Kamala_Harris',
 '/wiki/Patrick_Leahy',
 '/wiki/Chuck_Schumer',
 '/wiki/Dick_Durbin',
 '/wiki/Patty_Murray',
 '/wiki/Debbie_Stabenow',
 '/wiki/Mark_Warner',
 '/wiki/Elizabeth_Warren',
 '/wiki/Amy_Klobuchar',
 '/wiki/Bernie_Sanders',
 '/wiki/Joe_Manchin',
 '/wiki/Cory_Booker',
 '/wiki/Tammy_Baldwin',
 '/wiki/Gary_Peters',
 '/wiki/Catherine_Cortez_Masto',
 '/wiki/Jeff_Merkley',
 '/wiki/Brian_Schatz',
 '/wiki/Mitch_McConnell',
 '/wiki/John_Thune',
 '/wiki/John_Barrasso',
 '/wiki/Roy_Blunt',
 '/wiki/Joni_Ernst',
 '/wiki/Rick_Scott',
 '/wiki/Mike_Lee_(American_politician)',
 '/wiki/Mike_Crapo',
 '/wiki/Chuck_Grassley',
 '/wiki/Richard_Shelby',
 '/wiki/Tommy_Tuberville',
 '/wiki/Lisa_Murkowski',
 '/wiki/Dan_Sullivan_(U.S._senator)',
 '/wiki/Kyrsten_Sinema',
 '/wiki/Mark_Kelly',
 '/wiki/John_Boozman',
 '/wiki/Tom_Cotton',
 '/wiki/Dianne_Feinstein',
 '/wiki/Alex_Padilla',
 '/wiki/Michael_Bennet',
 '/wiki/John_Hickenlooper',
 '/wiki/Richard_Blumenthal',
 '/wiki/Chris_Murphy',
 '/wiki/To

In [18]:
all_senators_links = ['https://en.wikipedia.org'+ i for i in all_senators]

In [19]:
all_senators_links

['https://en.wikipedia.org/wiki/Kamala_Harris',
 'https://en.wikipedia.org/wiki/Patrick_Leahy',
 'https://en.wikipedia.org/wiki/Chuck_Schumer',
 'https://en.wikipedia.org/wiki/Dick_Durbin',
 'https://en.wikipedia.org/wiki/Patty_Murray',
 'https://en.wikipedia.org/wiki/Debbie_Stabenow',
 'https://en.wikipedia.org/wiki/Mark_Warner',
 'https://en.wikipedia.org/wiki/Elizabeth_Warren',
 'https://en.wikipedia.org/wiki/Amy_Klobuchar',
 'https://en.wikipedia.org/wiki/Bernie_Sanders',
 'https://en.wikipedia.org/wiki/Joe_Manchin',
 'https://en.wikipedia.org/wiki/Cory_Booker',
 'https://en.wikipedia.org/wiki/Tammy_Baldwin',
 'https://en.wikipedia.org/wiki/Gary_Peters',
 'https://en.wikipedia.org/wiki/Catherine_Cortez_Masto',
 'https://en.wikipedia.org/wiki/Jeff_Merkley',
 'https://en.wikipedia.org/wiki/Brian_Schatz',
 'https://en.wikipedia.org/wiki/Mitch_McConnell',
 'https://en.wikipedia.org/wiki/John_Thune',
 'https://en.wikipedia.org/wiki/John_Barrasso',
 'https://en.wikipedia.org/wiki/Roy_Blu

In [20]:
for i in all_senators_links:
    df = get_wiki_info(i)
    print(df)

                key          value
0         Full Name  Kamala Harris
11  Political party     Democratic
20         Children             NA
                key          value
0         Full Name  Patrick Leahy
20  Political party     Democratic
22         Children              3
                key          value
0         Full Name  Chuck Schumer
26  Political party     Democratic
28         Children              2
                key        value
0         Full Name  Dick Durbin
17  Political party   Democratic
19         Children    3[note 1]
                key         value
0         Full Name  Patty Murray
19  Political party    Democratic
21         Children             2
                key            value
0         Full Name  Debbie Stabenow
23  Political party       Democratic
25         Children                2
                key        value
0         Full Name  Mark Warner
15  Political party   Democratic
17         Children            3
                key             

                key        value
0         Full Name  Jerry Moran
12  Political party   Republican
14         Children            2
               key           value
0        Full Name  Roger Marshall
5  Political party      Republican
7         Children               4
                key            value
0         Full Name  Mitch McConnell
24  Political party       Republican
26         Children                3
               key       value
0        Full Name   Rand Paul
4  Political party  Republican
6         Children           3
                key         value
0         Full Name  Bill Cassidy
7   Political party    Republican
10         Children             3
                key                    value
0         Full Name             John Kennedy
8   Political party  Republican (since 2007)
11         Children                        1
                key          value
0         Full Name  Susan Collins
7   Political party     Republican
13         Children             NA


                key          value
0         Full Name  Patrick Leahy
20  Political party     Democratic
22         Children              3
                key                                              value
0         Full Name                                     Bernie Sanders
12  Political party  \nIndependent[a] (1978–present)\nDemocratic[b]...
15         Children                                                  1
                key        value
0         Full Name  Mark Warner
15  Political party   Democratic
17         Children            3
                key       value
0         Full Name   Tim Kaine
15  Political party  Democratic
17         Children           3
                key         value
0         Full Name  Patty Murray
19  Political party    Democratic
21         Children             2
                key           value
0         Full Name  Maria Cantwell
16  Political party      Democratic
21         Children              NA
                key                 

In [22]:
df[0]

,key,value
0,Full Name,Kamala Harris
11,Political party,Democratic
20,Children,NA


In [24]:
children = [i.loc[i.key == 'Children','value'].values[0] for i in df]
children

['NA',
 '3',
 '2',
 '3[note 1]',
 '2',
 '2',
 '3',
 '2, including Amelia',
 '1',
 '1',
 '3, including Heather',
 'NA',
 'NA',
 '3',
 'NA',
 '2',
 '2',
 '3',
 '2',
 '3',
 '4, including Matt',
 '1[3][4]',
 '2',
 '3',
 '5',
 '5',
 '2',
 '2',
 '2',
 '3',
 'NA',
 '2',
 '3',
 '2',
 'Katherine',
 '3',
 '3',
 '1',
 '4, including Matt Blumenthal',
 '2',
 '2',
 '3',
 '4',
 '2',
 '1',
 '2',
 '2',
 'NA',
 '5',
 '3',
 '3[note 1]',
 '2',
 '4',
 '4',
 '5',
 '1[3][4]',
 '2',
 '4',
 '3',
 '3',
 '3',
 '1',
 'NA',
 '5',
 '2',
 '3',
 '2, including Amelia',
 'NA',
 '2',
 '3',
 '1',
 '2',
 '3',
 '1',
 '4, including Matt',
 '3',
 '3',
 '4',
 '3',
 '3',
 'NA',
 '1',
 '3',
 '2',
 'Alicia and Rob1 stepdaughter',
 'NA',
 '2',
 'NA',
 '2',
 '2',
 '2',
 '2',
 '2',
 '5',
 '2',
 '3',
 '4[b]',
 '2',
 '5',
 '2',
 '4',
 '3',
 '1',
 '2',
 'NA',
 'NA',
 '2',
 '4',
 '2',
 '4',
 '2',
 '2',
 '3',
 '5, including Tagg',
 '3',
 '1',
 '3',
 '3',
 '2',
 'NA',
 '3, including Heather',
 '3, including Moore',
 '3',
 'NA',
 '3',
 '1

In [25]:
for item in children:
    new_list = []
    if item != '':
        new_list.append(item)
    print(new_list)

['NA']
['3']
['2']
['3[note 1]']
['2']
['2']
['3']
['2, including Amelia']
['1']
['1']
['3, including Heather']
['NA']
['NA']
['3']
['NA']
['2']
['2']
['3']
['2']
['3']
['4, including Matt']
['1[3][4]']
['2']
['3']
['5']
['5']
['2']
['2']
['2']
['3']
['NA']
['2']
['3']
['2']
['Katherine']
['3']
['3']
['1']
['4, including Matt Blumenthal']
['2']
['2']
['3']
['4']
['2']
['1']
['2']
['2']
['NA']
['5']
['3']
['3[note 1]']
['2']
['4']
['4']
['5']
['1[3][4]']
['2']
['4']
['3']
['3']
['3']
['1']
['NA']
['5']
['2']
['3']
['2, including Amelia']
['NA']
['2']
['3']
['1']
['2']
['3']
['1']
['4, including Matt']
['3']
['3']
['4']
['3']
['3']
['NA']
['1']
['3']
['2']
['Alicia and Rob1 stepdaughter']
['NA']
['2']
['NA']
['2']
['2']
['2']
['2']
['2']
['5']
['2']
['3']
['4[b]']
['2']
['5']
['2']
['4']
['3']
['1']
['2']
['NA']
['NA']
['2']
['4']
['2']
['4']
['2']
['2']
['3']
['5, including Tagg']
['3']
['1']
['3']
['3']
['2']
['NA']
['3, including Heather']
['3, including Moore']
['3']
['NA']
['3']
['1

1

In [35]:
import re
for i in new_list:
    children_number = (re.findall(r"^(\d+)", i))
children_number

['1']

In [36]:
political_party = [i.loc[i.key == 'Political party','value'].values[0] for i in df]
political_party

['Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic (1996–present)',
 'Democratic',
 '\nIndependent[a] (1978–present)\nDemocratic[b] (2015–2016, 2019–2020)\nLiberty Union (1970–1977)\n',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Republican',
 'Republican',
 'Republican',
 'Republican',
 'Republican',
 'Republican',
 'Republican',
 'Republican',
 'Republican',
 'Republican (1994–present)',
 'Republican',
 'Republican',
 'Republican',
 'Democratic (2004–present)',
 'Democratic (2018–present)[1]',
 'Republican',
 'Republican',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic (1988–present)',
 'Republican',
 'Republican',
 'Democratic',
 'Democratic',
 'Democratic',
 'Democratic',
 'Republican',
 'Republican',
 'Democratic',
 'Democratic',
 'Republican',
 'Republican (2012–present)',
 'Republi

In [37]:
political_party.count('Democratic')

59

In [38]:
political_party.count('Republican')

53

There are 59 senators in democratic party and 53 senators in repblican party.